## SEC FILINGS
### JOE-BUYDEM-DIPS!

v4 - 5/16/2022

SEE https://github.com/nss-data-science-cohort-5/sec_filings_for_real-joebuydemdips

In [1]:
# LIBRARY IMPORTS

from bs4 import BeautifulSoup as BS
import glob
# from IPython.core.display import display, HTML
from nltk import sent_tokenize, word_tokenize, \
    regexp_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import numpy as np
import pandas as pd
import re
import spacy
# from spacy import displacy   # displacy will eat all your RAM
from spacy.lang.en import English
from spacy.matcher import Matcher, PhraseMatcher
from spacy.symbols import nsubj, VERB
from spacy.tokens import Doc, Span

### FIRST GET TEXT FOR ALL 10k FILES

In [2]:
filepaths = glob.glob('../data/*.htm*')

In [3]:
# VAMSI'S CODE FROM 5/6/2022

ten_ks = {}

for filepath in filepaths:
    with open(filepath, 'r') as f:
        content = f.read()
        soup = BS(content, 
                  'html5lib')

        # Kill all script and style elements
        for script in soup(['script', 
                            'style']):
            script.extract()

        # Get text    
        text = soup.get_text()

        # Break into lines and remove leading and trailing space on each
        lines = (line.strip() \
                 for line in text.splitlines())

        # Break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines \
                  for phrase in line.split("  "))

        # Drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        # Append to dictionary
        ten_ks[filepath] = text

In [ ]:
# ten_ks[filepath][0:1000]

amrk = ten_ks['../data/amrk-10k_20210630.htm']

In [ ]:
# amrk[0:1000]

(MICHAEL SAYS NOT TO WORRY ABOUT 
REMOVING html TRASH AT THE BEGINNING, a la 
Tokenization_Practice.ipynb, 4/28/2022)

### TRYING SpaCy

TRYING FIRST ON annotations.json 
BEFORE MOVING ON TO html FILES.

DATACAMP PATTERNS - 
SEE https://www.datacamp.com/courses/advanced-nlp-with-spacy

In [3]:
annot_js = pd.read_json('../data/annotations.json')

In [4]:
annot_str = annot_js.to_string()

### FIRST TOKENIZE SENTENCES - SEE VAMSI'S NOTEBOOK

In [5]:
sent_doc = [sentence.lower() for sentence \
            in sent_tokenize(annot_str)]

In [6]:
sent_doc

["                    filing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [7]:
sentences = []
stemmer = PorterStemmer()
stock_stem = stemmer.stem('stock')
shares_stem = stemmer.stem('shares')

for idx, sentence in enumerate(sent_doc):
    if 'repurchase program' in sentence:
        chunk = sent_doc[idx-5:idx+6]
        for x in chunk:
            if stock_stem in x or shares_stem in x:
                sentences.append(chunk)
                break

In [8]:
sentences

[['we have remaining authority to repurchase 1,824,538 shares under this program, which has no expiration date.',
  'nan                  nan                            fiscal 2021         1,177,704     $32.9 million                  nan\n4       apog-20220226.html                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [9]:
nlp = spacy.load('en_core_web_sm')
values = []
for chunk in sentences:
    for sentence in chunk:
        text = nlp(sentence)
        for token in text:
            if token.like_num:
                #values.append(text[token.i-10:token.i+11])
                values.append(sentence)
                break
values

['we have remaining authority to repurchase 1,824,538 shares under this program, which has no expiration date.',
 'nan                  nan                            fiscal 2021         1,177,704     $32.9 million                  nan\n4       apog-20220226.html                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [22]:
# VAMSI HAD token.ent - THROWS AN ERROR

values[4]

'we have remaining authority to repurchase 1,824,538 shares under this program, which has no expiration date.'

### TO DO - 5/16/2022 - MICHAEL'S RECOMMENDATIONS --

#### SHOW repur_2 THE TOKENIZED SENTENCES

In [14]:
# annot_val = nlp(values)

In [34]:
# p_matcher = PhraseMatcher(nlp.vocab)

In [10]:
matcher = Matcher(nlp.vocab)

In [11]:
repur_1 = [{'LEMMA': 'repurchase'}]

In [12]:
matcher.add('REPURCHASE_PATTERN', [repur_1])

In [18]:
val_doc = []

for value in values:
    val_doc.append(nlp(value))

In [19]:
val_doc

[we have remaining authority to repurchase 1,824,538 shares under this program, which has no expiration date.,
 nan                  nan                            fiscal 2021         1,177,704     $32.9 million                  nan
 4       apog-20220226.html                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [15]:
values

['we have remaining authority to repurchase 1,824,538 shares under this program, which has no expiration date.',
 'nan                  nan                            fiscal 2021         1,177,704     $32.9 million                  nan\n4       apog-20220226.html                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [21]:
val_doc[0]

we have remaining authority to repurchase 1,824,538 shares under this program, which has no expiration date.

In [22]:
for ent in val_doc[1].ents:
    if ent.label_ == 'DATE':
        print(ent.text, ent.start_char,
              ent.end_char, ent.label_)

fiscal 2022 1131 1142 DATE


In [35]:
ent_cont = []

for doc in val_doc:
    ent_list = []
    for ent in doc.ents:
        if ent.label_ == 'CARDINAL':
            ent_list.append((ent.text, ent.start_char,
                  ent.end_char, ent.label_))
        if ent.label_ == 'DATE':
            ent_list.append((ent.text, ent.start_char,
                  ent.end_char, ent.label_))
        if ent.label_ == 'MONEY':
            ent_list.append((ent.text, ent.start_char,
                  ent.end_char, ent.label_))
    ent_cont.append({'text': doc.text, 'entities': ent_list})

In [27]:
ent_list

[('october 2020', 0, 12, 'DATE'),
 ('fiscal year ended july 31, 2021', 41, 72, 'DATE')]

In [36]:
ent_cont

[{'text': 'we have remaining authority to repurchase 1,824,538 shares under this program, which has no expiration date.',
  'entities': [('1,824,538', 42, 51, 'CARDINAL')]},
 {'text': 'nan                  nan                            fiscal 2021         1,177,704     $32.9 million                  nan\n4       apog-20220226.html                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [36]:
# GETTING ERROR "unhashable type: 'dict'"

# p_matcher.add('REPURCHASE_PATTERN', [repur_1])

#### MUST GET SPANS OF values?

MICHAEL SAYS, "It looks to me like values is a list of sentences. You'll probably want to iterate through that list, convert it to a Doc (by passing it through nlp ) and then apply the matcher to it."

In [40]:
# https://stackoverflow.com/questions/55675788/how-can-i-lemmatize-list-of-lists-in-python-using-spacy

annot_doc = spacy.tokens.Doc(nlp.vocab)
                             # vs = values) 

In [33]:
match_repur_1 = matcher(values)

ValueError: [E195] Matcher can be called on Doc or Span only, got list.

In [43]:
# CREATE Span OBJECT

for value in values:
    span = annot_doc[start:end]

NameError: name 'start' is not defined

In [42]:
# ITERATE OVER SPANS

lemmatized = []
# Iterate starting with 1
for index in range(1, len(values)):
    # Slice doc as described in the first point, so [0:3] and [3:6]
    span = annot_doc[values[index - 1] : values[index]]
    # Add lemmatized tokens as list to the outer list
    lemmatized.append([token.lemma_ for token in span])
lemmatized

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
# for ent in annot_sent.ents:
#    print(ent.text, ent.label_)

#### THEN ITERATE THROUGH TOKENIZED SENTENCES.
IS THERE A MATCH FOR THAT LEMMA?

In [ ]:
sentences = []
stemmer = PorterStemmer()
stock_stem = stemmer.stem('stock')
shares_stem = stemmer.stem('shares')

for idx, sentence in enumerate(sent_doc):
    if 'repurchase program' in sentence:
        chunk = sent_doc[idx-5:idx+6]
        for x in chunk:
            if stock_stem in x or shares_stem in x:
                sentences.append(chunk)
                break

#### LOOK FOR ent.label_ == 'DATE' --
USE A dict SO YOU WILL HAVE LABELS!

In [15]:
annot_sp = nlp(annot_str)

In [16]:
for ent in annot_sp.ents:
    if ent.label_ == 'DATE':
        print(ent.text, ent.start_char,
              ent.end_char, ent.label_)

authorization_number 1604 1624 DATE
November 10, 2020 2601 2618 DATE
2020 2816 2820 DATE
November 10, 2020 3115 3132 DATE
2020 4699 4703 DATE
fiscal 2022 4725 4736 DATE
fiscal 2022 4806 4817 DATE
fiscal 2004 6138 6149 DATE
fiscal 2022 6314 6325 DATE
fiscal 2021 7648 7659 DATE
fiscal 2020 7737 7748 DATE
fiscal 2022 9135 9146 DATE
fiscal 2021 9273 9284 DATE
fiscal 2020 9362 9373 DATE
fiscal 2004 10584 10595 DATE
fiscal 2022 10760 10771 DATE
fiscal 2021 10898 10909 DATE
fiscal 2020 10987 10998 DATE
fiscal 2004 11246 11257 DATE
December 2018 12386 12399 DATE
December 31, 2019 12593 12610 DATE
the year ended December 31, 2019 12619 12651 DATE
December 31, 2019 12844 12861 DATE
December 2018 12869 12882 DATE
December 2019 13994 14007 DATE
January 1, 2020 through December 31, 2020 14188 14229 DATE
March 2020 14234 14244 DATE
December 31, 2020 14469 14486 DATE
December 2019 14494 14507 DATE
December 2019 15619 15632 DATE
January 1, 2020 through December 31, 2020 15813 15854 DATE
March 2020 158

In [12]:
# annot_sent = nlp(sentences)

In [ ]:
# annot_sp = nlp(annot_str)

In [13]:
# for ent in annot_sent.ents:
#    print(ent.text, ent.label_)

In [14]:
# annot_sent

SEE Advanced NLP with SpaCy.txt LINES 199 - 214

In [13]:
matcher = Matcher(nlp.vocab)

In [14]:
repur_1 = [{'LEMMA': 'repurchase'}]

In [15]:
matcher.add('REPURCHASE_PATTERN', [repur_1])

In [18]:
match_repur_1 = matcher(annot_sp)

In [19]:
type(match_repur_1)

list

In [20]:
match_repur_1

[(7634494319942164669, 35, 36),
 (7634494319942164669, 43, 44),
 (7634494319942164669, 55, 56),
 (7634494319942164669, 81, 82),
 (7634494319942164669, 94, 95),
 (7634494319942164669, 190, 191),
 (7634494319942164669, 200, 201),
 (7634494319942164669, 240, 241),
 (7634494319942164669, 283, 284),
 (7634494319942164669, 311, 312),
 (7634494319942164669, 345, 346),
 (7634494319942164669, 365, 366),
 (7634494319942164669, 408, 409),
 (7634494319942164669, 436, 437),
 (7634494319942164669, 480, 481),
 (7634494319942164669, 490, 491),
 (7634494319942164669, 510, 511),
 (7634494319942164669, 553, 554),
 (7634494319942164669, 581, 582),
 (7634494319942164669, 626, 627),
 (7634494319942164669, 633, 634),
 (7634494319942164669, 663, 664),
 (7634494319942164669, 744, 745),
 (7634494319942164669, 751, 752),
 (7634494319942164669, 795, 796),
 (7634494319942164669, 854, 855),
 (7634494319942164669, 861, 862),
 (7634494319942164669, 905, 906),
 (7634494319942164669, 961, 962),
 (7634494319942164669, 1

In [22]:
pmm = nlp('We act as a principal market maker, maintaining a two-way market for buying and selling precious metals. This means we both sell product to and purchase product from our customers.')

In [23]:
matcher(pmm)

[]

In [25]:
cb = nlp("In April 2018, the Company's Board of Directors approved a share repurchase program which authorizes the Company to purchase up to 500,000 shares of its common stock from time to time, either in the open market or in block purchase transactions. The amount and timing of specific repurchases are subject to market conditions, applicable legal requirements, and other factors.  As of June 30, 2021, no shares had been repurchased under the program.")

In [26]:
matcher(cb)

[(7634494319942164669, 13, 14),
 (7634494319942164669, 51, 52),
 (7634494319942164669, 78, 79)]

In [35]:
# match_repur_1

In [37]:
# print('Repurchase matches:', [annot_sp[start:end].text \
#       for match_id, start, end in match_repur_1])

In [20]:
print('Total matches found:', len(match_repur_1))

Total matches found: 293


In [21]:
repur_2 = [{'POS': 'MONEY'}]

In [22]:
matcher.add('REPURCHASE_WITH_LOOT', [repur_2])

In [23]:
match_repur_2 = matcher(annot_sp)

In [24]:
print('Repurchase-with-loot matches:', 
      [annot_sp[start:end].text \
      for match_id, start, end in match_repur_2])

Repurchase-with-loot matches: ['repurchase', 'repurchase', 'Repurchases', 'repurchases', 'repurchases', 'repurchase', 'repurchased', 'repurchased', 'repurchased', 'repurchase', 'repurchased', 'repurchased', 'repurchased', 'repurchase', 'repurchase', 'repurchased', 'repurchased', 'repurchased', 'repurchase', 'repurchase', 'repurchase', 'repurchased', 'repurchase', 'repurchase', 'repurchase', 'repurchase', 'repurchase', 'repurchase', 'repurchased', 'repurchase', 'repurchased', 'repurchase', 'repurchase', 'repurchased', 'repurchase', 'repurchase', 'repurchase', 'repurchase', 'repurchased', 'repurchase', 'repurchase', 'repurchase', 'repurchase', 'repurchased', 'repurchase', 'repurchases', 'repurchase', 'repurchase', 'repurchase', 'repurchases', 'repurchases', 'repurchase', 'repurchase', 'repurchases', 'Repurchases', 'repurchase', 'repurchased', 'repurchase', 'repurchase', 'repurchases', 'repurchase', 'repurchase', 'repurchase', 'repurchase', 'repurchase', 'repurchase', 'repurchase', 'repur

#### NAVIGATING THE PARSE TREE

In [41]:
verbs = set()

In [42]:
for possible_subject in annot_sp:
    if possible_subject.dep == nsubj \
    and possible_subject.head.pos == VERB:
        verbs.add(possible_subject.head)

In [43]:
print(verbs)

{include, repurchased, purchased, authorized, repurchased, authorized, announced, entered, authorized, authorized, authorized, had, totaled, has, approved, authorizes, deems, repurchased, authorized, used, entered, repurchased, authorized, paid, discontinue, repurchased, had, have, repurchase, repurchased, announced, announced, expect, paid, repurchase, has, authorized, bbw20220129_10k.html, repurchase, repurchased, repurchased, repurchased, increased, authorized, represented, remained, approved, increased, authorized, repurchased, commence, authorized, have, make, utilized, repurchased, allowed, replaces, reduced, repurchased, repurchased, has, has, commenced, increased, authorized, authorized, repurchased, authorized, authorized, depend, repurchased, authorized, remaining, affect, authorized, repurchased, had, increased, reduce, authorized, entered, made, repurchase, have, expired, remained, purchased, increased, authorized, repurchased, authorize, repurchase, authorized, ended, auth

#### ITERATING AROUND THE LOCAL TREE

In [ ]:
span = annot_sp[annot_sp[4].left_edge.i : \
                annot_sp[4].right_edge.i + 1]

In [ ]:
with annot_sp.retokenize() as retok:
    retok.merge(span)

In [ ]:
for token in annot_sp:
    print(token.text, token.pos_, 
          token.dep_, token.head.text) 

In [ ]:
annot_sp.add_pipe('merge_entities')

In [ ]:
annot_sp.add_pipe('merge_noun_chunks')

In [ ]:
for annot_sp in nlp.pipe(annot_str):
    for token in annot_sp:
        if token.ent_type_ == 'MONEY':
            if token.dep_ in ('attr', 'dobj'):
                subj = [w for w in token.head.lefts \
                        if w.dep_ == 'nsubj']
                if subj:
                    print(subj[0], '-->', token)
            elif token.dep_ == 'pobj' \
            and token.head.dep_ == 'prep':
                print(token.head.head, '-->', token)

In [ ]:
# CAN DISABLE THE PARSER WITH

# nlp. = spacy.load('en_core_web_sm', disable = [parser])

### NAMED ENTITY RECOGNITION

#### NER 101

In [27]:
for ent in annot_sp.ents:
    print(ent.text, ent.start_char,
          ent.end_char, ent.label_)

authorization_number 1604 1624 DATE
November 10, 2020 2601 2618 DATE
Company's Board of Directors 2624 2652 ORG
$100 million 2666 2678 MONEY
Company 2726 2733 ORG
2020 2816 2820 DATE
Rule 10b5-1 2994 3005 ORG
Company 3061 3068 ORG
November 10, 2020 3115 3132 DATE
$100 million 3141 3153 MONEY
NaN               NaN               NaN                   3189 3246 ORG
NaN 3246 3249 MONEY
1 3250 3251 CARDINAL
601,822 4590 4597 CARDINAL
$30.8 million 4636 4649 MONEY
51.20 4683 4688 MONEY
2020 4699 4703 DATE
fiscal 2022 4725 4736 DATE
NaN                   4754 4775 ORG
NaN                             4775 4806 ORG
fiscal 2022 4806 4817 DATE
601,822 4828 4835 CARDINAL
$30.8 million 4840 4853 MONEY
NaN 4871 4874 MONEY
fiscal 2004 6138 6149 DATE
the Board of Directors 6151 6173 ORG
2,292,846 6272 6281 CARDINAL
fiscal 2022 6314 6325 DATE
$100.0 million 6347 6361 MONEY
NaN                   6379 6400 ORG
NaN                             6400 6431 ORG
2,292,846 6451 6460 CARDINAL
$100.0 million 6464 

2021 59891 59895 DATE
Repurchase Program 59896 59914 ORG
25 59999 60001 CARDINAL
$3.0 billion 60016 60028 MONEY
December 31, 2021 60050 60067 DATE
$0.8 billion 60091 60103 MONEY
NaN 60121 60124 MONEY
36 60125 60127 CARDINAL
January 24, 2022 61142 61158 DATE
an additional $2.0 billion 61244 61270 MONEY
2022 61328 61332 DATE
up to $4.2 billion 61400 61418 MONEY
2022 61456 61460 DATE
Repurchase Program 61461 61479 ORG
2021 61517 61521 DATE
Repurchase Program 61522 61540 ORG
January 24, 2022 61616 61632 DATE
$2.0 billion 61641 61653 MONEY
NaN               NaN               NaN                   61689 61746 ORG
NaN 61746 61749 MONEY
37 61750 61752 CARDINAL
the first quarter of 2021 62613 62638 DATE
Goldman Sachs & Co. 62718 62737 ORG
Goldman Sachs 62743 62756 ORG
$2.0 billion 62772 62784 MONEY
2018 62830 62834 DATE
Repurchase Program 62835 62853 ORG
$2.0 billion 62897 62909 MONEY
Goldman Sachs 62913 62926 ORG
5.9 million 62963 62974 CARDINAL
$1.7 billion 62992 63004 MONEY
$300 million 6307

NaN                   108754 108775 ORG
NaN                                     108775 108814 ORG
NaN               NaN 108814 108835 ORG
$3.7 billion 108841 108853 MONEY
NaN 108871 108874 MONEY
66 108875 108877 CARDINAL
January 28, 2020 110061 110077 DATE
Board of Directors 110083 110101 ORG
500 110116 110119 MONEY
the first quarter of 2020 110165 110190 DATE
12 months 110219 110228 DATE
March 2020 110238 110248 DATE
COVID-19 110289 110297 PERSON
257 110324 110327 MONEY
January 28, 2020 110366 110382 DATE
500 110400 110403 MONEY
March 2020 110432 110442 DATE
257 110475 110478 MONEY
NaN 110496 110499 MONEY
67 110500 110502 CARDINAL
October 2021 111610 111622 DATE
Board 111628 111633 ORG
up to $200 million 111710 111728 MONEY
Company 111736 111743 ORG
the three months ended November 28, 2021 111844 111884 DATE
3.4 million 111886 111897 CARDINAL
$88.4 million 111926 111939 MONEY
October 2021 111995 112007 DATE
$200 million 112016 112028 MONEY
three months ended November 28, 2021 112031 1

$200.0 million 157514 157528 MONEY
NaN               NaN               NaN                   157564 157621 ORG
NaN 157621 157624 MONEY
96 157625 157627 CARDINAL
October 2020 158700 158712 DATE
up to $200.0 million 158791 158811 MONEY
the fiscal year ended July 31, 2021 158852 158887 DATE
1,488,991 158904 158913 CARDINAL
109.17 158961 158967 MONEY
$162.5 million 159013 159027 MONEY
July 31, 2021 159035 159048 DATE
$37.5 million 159050 159063 MONEY
October 2020 159120 159132 DATE
$200.0 million 159139 159153 MONEY
fiscal year ended July 31, 2021 159161 159192 DATE
$162.5 million 159214 159228 MONEY


#### ACCESSING ENTITY ANNOTATIONS AND LABELS

In [ ]:
ents = [(e.text, e.start_char, e.end_char, e.label) \
        for e in annot_sp.ents]

In [ ]:
ent_1 = [annot_sp[0].text, annot_sp[0].ent_iob_, \
         annot_sp[0].ent_type_]

In [ ]:
ent_2 = [annot_sp[1].text, annot_sp[1].ent_iob_, \
         annot_sp[1].ent_type_]

In [ ]:
print(ent_1)

In [ ]:
print(ent_2)